# Part of Speech + Sentiment Analysis + Ngrams Selection

In this file, we extract verbs and adjectives out from each review entry and get sentiment analysis of them. We use count vectorizer to build feacture matrix base on these verbs and adjectives with ngrams function. Then we weight them with their sentiment scores before we finally give the data to SVC model to train.

## Load Data

In [28]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


train = pd.read_csv('../Lemmatization/result_train.csv')
test = pd.read_csv('../Lemmatization/result_test.csv')


## Data Preprocessing

After data has been lemmatized, some of the entries may contain nothing anymore and this could potentially cause troubles difficult for debugging later. So we are using fillna to filter all the nan entries


In [ ]:
train.fillna(" ", inplace = True)
test.fillna(" ", inplace = True)


In [29]:
print train.shape
print test.shape


(156060, 3)
(66292, 2)


Since we are thinking of making an ensemble model in the end, we split the known data into two groups. One for the current model trainning and the another for the ensemble model to analyze how accurate this model is. By comparing the ensemble knows how to assign individual model weights.

In [ ]:
# train = train[:10]
# test = test[:10]

split_indexs = int(len(train)/2)
train_test = train[: split_indexs]
train_train = train[split_indexs:]
columns = ['PhraseId', 'Phrase', 'Sentiment']
train_test= pd.DataFrame(train_test, columns=columns)
train_train= pd.DataFrame(train_train, columns=columns)


We combind all the train and test data together. This is prepared to find the same unique words feature vectors from the vectorization function provided by sklearn. 

In [30]:

train = np.array(train)
test = np.array(test)

# train_sentiment = train[:,-1]
train = train[:, :2]

print train.shape
print test.shape

data = np.concatenate((train, test), axis=0)
print data.shape
columns = ['Phrase', 'PhraseId']
data= pd.DataFrame(data, columns=columns)
test = pd.DataFrame(test, columns=columns)



(156060, 2)
(66292, 2)
(222352, 2)


## Exract adj and vebvers

In a sentence, generally the verbs and the adjectives are the most important words in that sentence to carry out the sentiment feeling and meaning of the whole sentence. So here we think of using textblob library to extract adjectives and verbs and analyze their sentiments individually.

In [ ]:
from textblob import TextBlob
def extract_verbs_and_adj(phrase):
    words = []
    
    tags = TextBlob(phrase).tags
    for tag in tags:
        if tag[1][:2] == "VB" or tag[1][:2] == "JJ":
            words.append(tag[0])
    return words

def looping_extract_verbs_and_adj(data_set):
    phrases_vb_adj = []

    for (index, phrase) in enumerate(data_set):
        if index % 10 == 0:
            print index
        phrases_vb_adj.append(' '.join(extract_verbs_and_adj(phrase)))
    return phrases_vb_adj

data_vb_adj = looping_extract_verbs_and_adj(data.Phrase)



0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160

Use the count vectorizer to get the unique word matrix of the whole data set including both train and test data.
Here, we specify ngram_range=(0, 4) parameter for the countvectorizer so it will select all combinations among verbs and adjectives for each review entry ranging from 1 to 4 units.

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

v1 = CountVectorizer(ngram_range=(0, 4),  min_df=2, max_df=0.95, max_features=1000)
v1.fit(data_vb_adj)


train_test_vb_adj = looping_extract_verbs_and_adj(train_test.Phrase)
train_train_vb_adj = looping_extract_verbs_and_adj(train_train.Phrase)
test_vb_adj = looping_extract_verbs_and_adj(test.Phrase)


0
0
0


After extracting the verbs and adjectives, we want to weight the frequencies of the words by their individual sentiments.

In [11]:
def add_vb_adj_sentiment(phrases_vb_adj):
    X2 = v1.transform(phrases_vb_adj)

    sentiments = []
    for feature in v1.get_feature_names():
        sentiment = TextBlob(feature).sentiment
        value = sentiment.polarity * sentiment.subjectivity
        sentiments.append(value)
    from scipy.sparse import csr_matrix

    X3 = csr_matrix(X2.shape)
    for index in range(X2.shape[0]):
        row = X2[index]
        for col in range(row.shape[1]):
            X3[index, col] = X2[index, col] * sentiments[col]

    return X3

X_train_test = add_vb_adj_sentiment(train_test_vb_adj)
X_train_train = add_vb_adj_sentiment(train_train_vb_adj)
X_test = add_vb_adj_sentiment(test_vb_adj)

/home/zhecan/anaconda2/lib/python2.7/site-packages/scipy/sparse/compressed.py:730: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [12]:

print X_train_train.shape
print X_train_test.shape
print X_test.shape

(5, 9)
(5, 9)
(10, 9)


## Trainning Model

We let the support vector classifier model to be trained base on the train_train data set and make predictions on both the train_test and test data sets. So later the ensemble model will use the train_test data set's prediction to determine how accurate this model is and then assign appropriate weights for its predictions on test data set for final combining.

In [21]:
from sklearn.svm import SVC
clf = SVC()
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

# y = np.array([1,2,3,4,5])
clf.fit(X_train_train, train_train.Sentiment) 

y_train_test_predict = clf.predict(X_train_test)
y_test_predict = clf.predict(X_test)


train_output = pd.DataFrame({
    'PhraseId': train_test.PhraseId,
    'Predicted': y_train_test_predict,
    'Sentiment': train_test.Sentiment
})



test_output = pd.DataFrame({
    'PhraseId': test.PhraseId,
    'Sentiment': y_test_predict
})

5
<class 'pandas.core.series.Series'>
